In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../../companies/src/')

In [4]:
import company_info as ci
import opencorporates as oc
from glob import glob
import pandas as pd
from followthemoney import model
from alephclient.api import AlephAPI
import json
import os
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [5]:
PATH = os.environ.get('PATH_FISHERIES')

In [6]:
concerns = []
for file in glob(PATH + 'companies/raw_concern/*.html'):
    concern = ci.get_concern_html(file)
    concerns.append(concern)
    
concern = pd.concat(concerns).reset_index(drop=True)

In [7]:
deelnemingen = []
for file in glob(PATH + 'companies/raw/*.har'):
    deelneming = ci.parse_company_info(file, 'participations')
    deelnemingen.append(deelneming)

deelnemingen = pd.concat(deelnemingen)
deelnemingen.drop_duplicates(subset=['country', 'name', 'year', 'percentage'], inplace=True)
deelnemingen.dropna(subset='name', inplace=True)
deelnemingen = deelnemingen[deelnemingen.name != '--'].copy()
deelnemingen.reset_index(drop=True, inplace=True)

In [9]:
concern.to_csv(PATH + 'concern_lords.csv', index=False)
deelnemingen.to_csv(PATH + 'participations_lords.csv', index=False)

In [ ]:
concern.head()

In [ ]:
deelnemingen.sample(10)

In [ ]:
result = oc.search_company('Oderbank Hochseefischerei GmbH', 'de')

In [ ]:
result

In [ ]:
person = {'start_index': 0, 'total_results': 9, 'items': [{'occupation': 'Director', 'nationality': 'Dutch', 'officer_role': 'director', 'country_of_residence': 'Netherlands', 'appointed_on': '2006-01-31', 'name_elements': {'forename': 'Jan', 'surname': 'VAN DER PLAS', 'other_forenames': 'Cornelis'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_name': 'ARMANA LIMITED', 'company_status': 'active', 'company_number': '02114924'}, 'links': {'company': '/company/02114924'}, 'address': {'premises': 'Parlevliet & Van Der Plas', 'address_line_1': 'Voorschoterweg 31', 'country': 'Netherlands', 'locality': '2235 Se Volkenburg Zh'}}, {'officer_role': 'director', 'country_of_residence': 'Netherlands', 'nationality': 'Dutch', 'occupation': 'Director', 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_name': 'SWANELLA LIMITED', 'company_status': 'active', 'company_number': '02980262'}, 'links': {'company': '/company/02980262'}, 'address': {'locality': '2235 Se Valkenburg Zh', 'country': 'Netherlands', 'address_line_1': 'Voorschoterweg 31', 'premises': 'Parlevliet & Van Der Plas'}, 'name_elements': {'forename': 'Jan', 'surname': 'VAN DER PLAS', 'other_forenames': 'Cornelis'}, 'appointed_on': '2006-01-31'}, {'links': {'company': '/company/02114161'}, 'address': {'premises': 'Parlevliet & Van Der Plas', 'address_line_1': 'Voorschoterweg 31', 'country': 'Netherlands', 'locality': '2235 Se Valkenburg Zh'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_name': 'JACINTA LIMITED', 'company_status': 'active', 'company_number': '02114161'}, 'appointed_on': '2006-01-31', 'name_elements': {'surname': 'VAN DER PLAS', 'forename': 'Jan', 'other_forenames': 'Cornelis'}, 'country_of_residence': 'Netherlands', 'officer_role': 'director', 'nationality': 'Dutch', 'occupation': 'Director'}, {'occupation': 'Director', 'officer_role': 'director', 'country_of_residence': 'Netherlands', 'nationality': 'Dutch', 'appointed_on': '2006-01-31', 'name_elements': {'other_forenames': 'Cornelis', 'surname': 'VAN DER PLAS', 'forename': 'Jan'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_number': '00054268', 'company_status': 'active', 'company_name': 'J. MARR (FISHING) LIMITED'}, 'address': {'locality': '2235 Se Valkenburg Zh', 'country': 'Netherlands', 'address_line_1': 'Voorschoterweg 31', 'premises': 'Parlevliet & Van Der Plas'}, 'links': {'company': '/company/00054268'}}, {'occupation': 'Director', 'country_of_residence': 'Netherlands', 'officer_role': 'director', 'nationality': 'Dutch', 'name_elements': {'forename': 'Jan', 'surname': 'VAN DER PLAS', 'other_forenames': 'Cornelis'}, 'appointed_on': '2006-01-31', 'links': {'company': '/company/03817237'}, 'address': {'country': 'Netherlands', 'locality': '2235 Se Valkenburg Zh', 'premises': 'Parlevliet & Van Der Plas', 'address_line_1': 'Voorschoterweg 31'}, 'appointed_to': {'company_name': 'KIRKELLA LIMITED', 'company_status': 'active', 'company_number': '03817237'}, 'name': 'Jan Cornelis VAN DER PLAS'}, {'appointed_on': '2006-01-18', 'name_elements': {'other_forenames': 'Cornelis', 'forename': 'Jan', 'surname': 'VAN DER PLAS'}, 'appointed_to': {'company_status': 'active', 'company_name': 'MARR MANAGEMENT LIMITED', 'company_number': '05647289'}, 'name': 'Jan Cornelis VAN DER PLAS', 'address': {'locality': '2235 Se Valkenburg Zh', 'country': 'Netherlands', 'address_line_1': 'Voorschoterweg 31', 'premises': 'Parlevliet & Van Der Plas'}, 'links': {'company': '/company/05647289'}, 'occupation': 'Director', 'officer_role': 'director', 'country_of_residence': 'Netherlands', 'nationality': 'Dutch'}, {'name_elements': {'forename': 'Jan', 'surname': 'VAN DER PLAS', 'other_forenames': 'Cornelis'}, 'appointed_on': '2004-09-01', 'address': {'locality': '2235 Se Valkenburg Zh', 'country': 'Netherlands', 'address_line_1': 'Voorschoterweg 31', 'premises': 'Parlevliet & Van Der Plas'}, 'links': {'company': '/company/05219340'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_name': 'UK FISHERIES LIMITED', 'company_status': 'active', 'company_number': '05219340'}, 'occupation': 'Managing Director', 'country_of_residence': 'Netherlands', 'officer_role': 'director', 'nationality': 'Dutch'}, {'name_elements': {'other_forenames': 'Cornelis', 'surname': 'VAN DER PLAS', 'forename': 'Jan'}, 'appointed_on': '2004-02-15', 'address': {'country': 'Netherlands', 'locality': '2235 Se Valkenburg Zh', 'premises': 'Parlevliet & Van Der Plas', 'address_line_1': 'Voorschoterweg 31'}, 'links': {'company': '/company/02019266'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_status': 'active', 'company_name': 'LIONMAN LIMITED', 'company_number': '02019266'}, 'occupation': 'Managing Director', 'country_of_residence': 'Netherlands', 'officer_role': 'director', 'nationality': 'Dutch'}, {'name_elements': {'other_forenames': 'Cornelis', 'forename': 'Jan', 'surname': 'VAN DER PLAS'}, 'appointed_on': '2004-02-15', 'address': {'locality': '2235 Se Valkenburg Zh', 'country': 'Netherlands', 'address_line_1': 'Voorschoterweg 31', 'premises': 'Parlevliet & Van Der Plas'}, 'links': {'company': '/company/01501627'}, 'name': 'Jan Cornelis VAN DER PLAS', 'appointed_to': {'company_status': 'active', 'company_name': 'BOYD LINE LIMITED', 'company_number': '01501627'}, 'occupation': 'Director', 'country_of_residence': 'Netherlands', 'officer_role': 'director', 'nationality': 'Dutch'}], 'name': 'Jan Cornelis VAN DER PLAS', 'links': {'self': '/officers/zlruz9CLpxkWxbgPguBg96_T3HA/appointments'}, 'etag': '3f295231070b65a14e8bb86652fc396afe2e15b5', 'date_of_birth': {'month': 4, 'year': 1954}, 'is_corporate_officer': False, 'kind': 'personal-appointment', 'items_per_page': 35}


In [ ]:
dicts = []
pers_dict = {}

keys = [key for key in person.keys() if key not in ['items', 'start_index', 'items_per_page', 'total_results']]

for key in keys:
    pers_dict.update({key: person.get(key)})

for p in person.get('items'):
    pers_dict.update({'name': p.get('name'),
                 'first_name': p.get('name_elements').get('forename'),
                 'last_name': p.get('name_elements').get('surname'),
                 'nationality': p.get('nationality'),
                 'country_of_residence': p.get('country_of_residence'),
                 'company_name': p.get('appointed_to').get('company_name'),
                 'company_status': p.get('appointed_to').get('company_status'),
                 'company_number': p.get('appointed_to').get('company_number'),
                 'role': p.get('officer_role'),
                 'occupation': p.get('occupation'),
                 'appointed_on': p.get('appointed_on'),
                 'premises': p.get('address').get('premises'),
                 'address_line1': p.get('address').get('address_line_1'),
                 'addres_line2': p.get('address').get('address_line_2'),
                 'country': p.get('address').get('country'),
                 'locality': p.get('address').get('locality')})
    dicts.append(pers_dict)

benoemingen = pd.DataFrame(dicts)

if len(benoemingen) != person.get('total_results'):
    raise ValueError('not all records parsed')
    
                 
    
    

In [ ]:
pd.DataFrame(person.get('items')).get('address')[1]